In [5]:
import numpy as np
import matplotlib.pyplot as plt
#from pydgrid.plot_bokeh import plot_results
import sympy as sym
import pydae.build as db
import pydae.build_cffi as db

from pydae.grid_urisi import unb_ri_si
import json
import networkx as nx
import time

In [6]:
grid = unb_ri_si('grid_trafo_2bus.json')

In [7]:
x_dummy,u_dummy = sym.symbols('x_dummy,u_dummy', real=True)
params_dict  = grid.dae['params']
f_list = [u_dummy - x_dummy]
x_list = [x_dummy]
g_list = grid.dae['g'] 
y_list = grid.dae['y'] 
u_dict = grid.dae['u']
u_dict.update({'u_dummy':0.0})

h_dict = grid.dae['h_dict']
h_dict.update(grid.dae['h_v_m_dict'])
h_dict.update({'z_dummy':0.5*u_dummy - x_dummy})


sys_dict = {'name':'grid_trafo_2bus',
           'params_dict':params_dict,
           'f_list':f_list,
           'g_list':g_list,
           'x_list':x_list,
           'y_ini_list':y_list,
           'y_run_list':y_list,
           'u_run_dict':u_dict,
           'u_ini_dict':u_dict,
           'h_dict':h_dict
           }

db.build(sys_dict,verbose=True);


check_system (time: 0.0)
computing jacobians Fx_run,Fy_run  (time: 0.057 s)
computing jacobians Gx_run,Gy_run  (time: 0.062 s)
computing jacobians Fu_run,Gu_run  (time: 0.924 s)
computing jacobians Fx_ini,Fy_ini  (time: 1.111 s)
computing jacobians Gx_ini,Gy_ini  (time: 1.116 s)
computing jacobians Hx_run,Hy_run,Hu_run  (time: 1.6057071685791016 s)
end system  (time: 3.266 s)
computing jac_ini (time: 0.0)
computing jac_run (time: 0.019945144653320312)
computing jac_trap (time: 0.038603782653808594)
end of jacobians computation (time: 0.063)
writting f_ini and g_ini code (time: 0.000 s)
writting f_run and g_run code (time: 1.024 s)
writting h_run code (time: 1.928 s)
converting jac_ini to sp_jac_ini  (time: 2.528 s)
running sym2rhs for sp_jac_ini (time: 2.545 s)
converting jac_run to sp_jac_run  (time: 3.208 s)
running sym2rhs for sp_jac_run (time: 3.217 s)
converting jac_trap to sp_jac_trap  (time: 3.742 s)
running sym2rhs for sp_jac_trap (time: 3.752 s)
wrtting  de_jac_trap code (time

In [11]:
sys_dict['f']

Matrix([[u_dummy - x_dummy]])

In [24]:
from sympy.matrices.sparsetools import _doktocsr
from sympy import SparseMatrix
from sympy.codegen.ast import Assignment
from scipy.sparse import csr_matrix,save_npz
from pydae.build_cffi import sym2rhs2,rhs2str,str2src,sym2xyup

In [31]:
def sym2rhs2(data,indices,indptr,shape,sys,inirun):


    rhs_list = []
    
    string_full_sym = ''
    
    for irow in range(len(indptr)-1):
        for k in range(indptr[irow],indptr[irow+1]):
            icol = indices[k]
            data_k = data[k]
            if not data[k] == 0:
                string_sym = sym.ccode(sym.N(data_k)) + ';\n#'
                string_full_sym += string_sym
    

    string_full = sym2xyup(string_full_sym,sys,inirun)
    
    string_split = string_full.split('#')  
    for irow in range(len(indptr)-1):
        for k in range(indptr[irow],indptr[irow+1]):
            icol = indices[k]
            data_k = string_split[k]
            if not data_k == 0:
                string_i = data_k
                
                tipo = 'num'
                if 'x[' in string_i or 'y[' in string_i:
                    tipo = 'xy' 
                elif 'p[' in string_i or 'u[' in string_i or 'Dt' in string_i:
                    tipo = 'up' 
               
                rhs_list += [(string_i,tipo,irow,icol)]

                
    return rhs_list

In [32]:
verbose = True
t_0 = time.time()
sys = sys_dict
jac_num=True
 

## Fu_run and Gu_run ###############################################################
defs_de_uz   = ''
source_de_uz = ''
defs_sp_uz   = ''
source_sp_uz = ''

for item in ['Fu_run','Gu_run','Hx_run','Hy_run','Hu_run']:
    if verbose: print(f'converting {item}  (time: {time.time()-t_0:0.3f} s)')  
    print(len(SparseMatrix(sys[item])))
    sp_jac_list = _doktocsr(SparseMatrix(sys[item]))  
    data = sp_jac_list[0]
    indices = sp_jac_list[1]
    indptr = sp_jac_list[2]
    shape = sp_jac_list[3]

    sp_jac_num_matrix = csr_matrix((np.zeros(len(data)),indices,indptr),shape=shape)


    if verbose: print(f'running sym2rhs for {item} (time: {time.time()-t_0:0.3f} s)')
    rhs_list = sym2rhs2(data,indices,indptr,shape,sys,'run')       

    if verbose: print(f'wrtting  {item} code (time: {time.time()-t_0:0.3f} s)')   
    string_xy,string_up,string_num = rhs2str(rhs_list,'out',sp_jac_num_matrix.data,shape,mode='dense')

    if jac_num:
        defs_de_jac,source_de_jac = str2src(f'de_{item}',string_xy,string_up,string_num,matrix_name='out')
    else:
        defs_de_jac,source_de_jac = str2src(f'de_{item}',string_xy,string_up,'\n',matrix_name='out')


    if verbose: print(f'writting {item} code (time: {time.time()-t_0:0.3f} s)')      
    string_xy,string_up,string_num = rhs2str(rhs_list,'out',sp_jac_num_matrix.data,shape,mode='crs')
    if jac_num:
        defs_sp_jac,source_sp_jac = str2src('sp_{item}',string_xy,string_up,string_num,matrix_name='out')
    else:
        defs_sp_jac,source_sp_jac = str2src('sp_{item}',string_xy,string_up,'\n',matrix_name='out')

    defs_de_uz   += defs_de_jac
    source_de_uz += source_de_jac
    defs_sp_uz   += defs_sp_jac
    source_sp_uz += source_sp_jac

    save_npz(f"{sys['name']}_{item}_num.npz",sp_jac_num_matrix, compressed=True)


converting Fu_run  (time: 0.000 s)
35
running sym2rhs for Fu_run (time: 0.001 s)
wrtting  Fu_run code (time: 0.004 s)
writting Fu_run code (time: 0.004 s)
converting Gu_run  (time: 0.009 s)
3220
running sym2rhs for Gu_run (time: 0.012 s)
wrtting  Gu_run code (time: 0.049 s)
writting Gu_run code (time: 0.049 s)
converting Hx_run  (time: 0.052 s)
66
running sym2rhs for Hx_run (time: 0.052 s)
wrtting  Hx_run code (time: 0.055 s)
writting Hx_run code (time: 0.055 s)
converting Hy_run  (time: 0.057 s)
6072
running sym2rhs for Hy_run (time: 0.063 s)
wrtting  Hy_run code (time: 0.294 s)
writting Hy_run code (time: 0.296 s)
converting Hu_run  (time: 0.301 s)
2310
running sym2rhs for Hu_run (time: 0.304 s)
wrtting  Hu_run code (time: 0.361 s)
writting Hu_run code (time: 0.362 s)


In [4]:
h_dict

{'i_t_AC1_AC2_1_a_r': 0.0392156862745098*v_AC1_a_i + 0.00980392156862745*v_AC1_a_r - 0.0196078431372549*v_AC1_b_i - 0.00490196078431373*v_AC1_b_r - 0.0196078431372549*v_AC1_c_i - 0.00490196078431373*v_AC1_c_r - 1.69808902702831*v_AC2_a_i - 0.424522256757078*v_AC2_a_r + 1.69808902702831*v_AC2_b_i + 0.424522256757078*v_AC2_b_r,
 'i_t_AC1_AC2_1_a_i': 0.00980392156862745*v_AC1_a_i - 0.0392156862745098*v_AC1_a_r - 0.00490196078431373*v_AC1_b_i + 0.0196078431372549*v_AC1_b_r - 0.00490196078431373*v_AC1_c_i + 0.0196078431372549*v_AC1_c_r - 0.424522256757078*v_AC2_a_i + 1.69808902702831*v_AC2_a_r + 0.424522256757078*v_AC2_b_i - 1.69808902702831*v_AC2_b_r,
 'i_t_AC1_AC2_1_b_r': -0.0196078431372549*v_AC1_a_i - 0.00490196078431373*v_AC1_a_r + 0.0392156862745098*v_AC1_b_i + 0.00980392156862745*v_AC1_b_r - 0.0196078431372549*v_AC1_c_i - 0.00490196078431373*v_AC1_c_r - 1.69808902702831*v_AC2_b_i - 0.424522256757078*v_AC2_b_r + 1.69808902702831*v_AC2_c_i + 0.424522256757078*v_AC2_c_r,
 'i_t_AC1_AC2_1